In [ ]:
!unzip "/content/drive/MyDrive/BTP 1/Ashwin/original/augmented_dataset.zip" -d "/content/augmented_dataset"

In [ ]:
train_csv_path="/content/drive/MyDrive/BTP 1/Ashwin/original/train.csv"
valid_csv_path="/content/drive/MyDrive/BTP 1/Ashwin/original/valid.csv"

In [ ]:
# bpanet_eyesdefy.py
"""
Simplified BPANet implementation for conjunctiva-only (EYES-DEFY).
- ResNet50 backbone
- CBAM-style Channel+Spatial Attention (inserted between layer1 and layer2)
- Morphological opening preprocessing (3x3 kernel, 2 iterations)
- Dual-output: regression (Hb) and classification (anemia vs non-anemia)
Adjust dataset paths and hyperparams as needed.
"""

import os
import random
import math
from typing import Tuple

import cv2
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
import torchvision.models as models
from torchvision.models import ResNet50_Weights

# -------------------------
# Config / Hyperparameters
# -------------------------
IMG_SIZE = 224
BATCH_SIZE = 16
NUM_EPOCHS = 140
LR = 1e-4
WEIGHT_DECAY = 1e-5
# LAMBDA_REG = 1.0   # weight for regression MSE
# LAMBDA_CLS = 1.0   # weight for classification CE
# ANEMIA_THRESHOLD = 12.0  # g/dL; adjust if you prefer sex-specific thresholds
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ======================
# Dual Loss Bin Settings
# ======================
X_MIN = 7.0
X_MAX = 17.4
N_BINS = 48
BIN_SIZE = (X_MAX - X_MIN) / N_BINS

# Bin centers (for expected value)
BIN_CENTERS = torch.linspace(X_MIN + BIN_SIZE/2, X_MAX - BIN_SIZE/2, N_BINS).to(DEVICE)

# -------------------------
# Utils: seeds
# -------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

##Preprocessing

##Dataset

In [ ]:
# -------------------------
# Dataset
# -------------------------
class EyesDefyDataset(Dataset):
    def __init__(self, csv_file: str, transform=None):
        """
        csv_file: CSV with columns ['image_path','hb'] where image_path is absolute or relative path to image.
        transform: torchvision transforms to apply after preprocessing
        """
        df = pd.read_csv(csv_file)
        # Basic cleaning: drop missing
        df = df.dropna(subset=['image_path', 'Hgb']).reset_index(drop=True)
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image_path']
        hb_val = float(row["Hgb"])

        # Compute bin index safely
        bin_idx = math.floor((hb_val - X_MIN) / BIN_SIZE)
        bin_idx = max(0, min(N_BINS - 1, bin_idx))
        bin_idx = torch.tensor(bin_idx, dtype=torch.long)

        hb = torch.tensor(hb_val, dtype=torch.float32)

        # Read with OpenCV -> BGR
        img_bgr = cv2.imread(img_path)
        if img_bgr is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        # Convert to RGB PIL for torchvision transforms
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        pil = Image.fromarray(img_rgb)
        if self.transform is None:
          raise ValueError("transform must be provided to EyesDefyDataset")
        img_tensor = self.transform(pil)

        age = row["Age"]
        gender = row["Gender"]
        # region = row["Region"]

        demographic = torch.tensor([age, gender], dtype=torch.float32)

        return img_tensor,demographic, hb,bin_idx

##CSA

In [ ]:
# -------------------------
# CBAM-like module (Channel + Spatial attention)
# -------------------------
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.mlp = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_planes // reduction, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.mlp(self.avg_pool(x))
        max_out = self.mlp(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        assert kernel_size in (3,7)
        padding = 3 if kernel_size == 7 else 1
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # along channel: compute max and avg
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        avg_out = torch.mean(x, dim=1, keepdim=True)
        cat = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(cat)
        return self.sigmoid(out)

class ChannelSpatialAttention(nn.Module):
    def __init__(self, in_planes, reduction=16, spatial_kernel=7):
        super().__init__()
        self.channel_att = ChannelAttention(in_planes, reduction=reduction)
        self.spatial_att = SpatialAttention(kernel_size=spatial_kernel)

    def forward(self, x):
        ca = self.channel_att(x) * x
        sa = self.spatial_att(ca) * ca
        return sa

##Backbone

In [ ]:
# -------------------------
# Backbone with CSA inserted between layer1 and layer2 of ResNet50
# -------------------------
class ResNet50WithCSA(nn.Module):
    def __init__(self, pretrained=True, feature_dim=1000):
        super().__init__()
        resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        # Keep early layers
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1                                       # Stage1:low-level features
        self.csa = ChannelSpatialAttention(in_planes=256, reduction=16)   # Insert CSA here (stage1 output = 256 channels)
        self.layer2 = resnet.layer2                                       # Stage2:higher layers
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        # pooling and final projection
        self.avgpool = resnet.avgpool  # adaptive avg pool
        # Project to feature_dim
        self.feature_dim = feature_dim
        self.fc_proj = nn.Linear(resnet.fc.in_features, feature_dim)

    def forward(self, x):
        x = self.conv1(x)   # /2
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x) # /2
        x = self.layer1(x)
        x = self.csa(x)     # channel-spatial attention applied early
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)  # (B, C, 1, 1)
        x = torch.flatten(x, 1)
        feat = self.fc_proj(x)  # (B, feature_dim)
        return feat


In [ ]:
# -------------------------
# BPANet simplified model (single branch)
# -------------------------
class BPANet_Simple(nn.Module):
    def __init__(self, pretrained=True, feature_dim=1000):
        super().__init__()
        self.backbone = ResNet50WithCSA(pretrained=pretrained, feature_dim=feature_dim)
        # optionally you could include demographic embeddings here
        # Head:
        self.reg_bins_head = nn.Linear(feature_dim+2, N_BINS)#doubt

    def forward(self, x, demographic):
        feat = self.backbone(x)                     # (B, 512)
        fused = torch.cat([feat, demographic], dim=1)   # (B, 515)
        bin_logits = self.reg_bins_head(fused)   # (B, N_BINS)
        prob = torch.softmax(bin_logits, dim=1)    # (B, 48)
        hb_pred = torch.sum(prob * BIN_CENTERS.to(bin_logits.device), dim=1)  # ensure device match
        hb_pred = hb_pred.squeeze()  # (B,)
        return hb_pred, bin_logits



In [ ]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=1.0, gamma=2.0):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.ce = nn.CrossEntropyLoss(reduction='none')

#     def forward(self, logits, target):
#         ce_loss = self.ce(logits, target)     # shape (B,)
#         pt = torch.exp(-ce_loss)
#         focal_loss = self.alpha * (1 - pt)**self.gamma * ce_loss
#         return focal_loss.mean()
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, target):
        # logits: (B, C), target: (B,) long
        probs = torch.softmax(logits, dim=1)          # (B, C)
        pt = probs.gather(1, target.unsqueeze(1)).squeeze(1)  # (B,)
        ce_loss = -torch.log(pt + 1e-12)              # stable
        focal = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal.mean() if self.reduction == 'mean' else focal.sum()

# ====================
# Dual Loss (Training)
# ====================
focal_loss_fn = FocalLoss()
mse_loss = nn.MSELoss()
ALPHA = 0.5

In [ ]:
# -------------------------
# Training
# -------------------------
def train_one_epoch(model, loader, optimizer, criterion_reg, criterion_focal, device):
    model.train()
    running_loss = 0.0
    all_preds_hb = []
    all_gt_hb = []
    for imgs, demographic, hb_vals, bin_idx in tqdm(loader, desc="train", leave=False):
        imgs = imgs.to(device)
        demographic = demographic.to(device)
        hb_vals = hb_vals.to(device)
        bin_idx = bin_idx.to(device).long()

        optimizer.zero_grad()
        hb_pred, bin_logits = model(imgs, demographic)

        loss_reg = criterion_reg(hb_pred, hb_vals)
        loss_bin = criterion_focal(bin_logits, bin_idx)

        loss = loss_reg + ALPHA * loss_bin
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

        all_preds_hb.extend(hb_pred.detach().cpu().numpy().tolist())
        all_gt_hb.extend(hb_vals.detach().cpu().numpy().tolist())

    epoch_loss = running_loss / len(loader.dataset)
    # metrics
    mae = mean_absolute_error(all_gt_hb, all_preds_hb)
    return epoch_loss, {"mae":mae}


In [ ]:
def eval_model(model, loader, criterion_reg, criterion_focal, device):
    model.eval()
    running_loss = 0.0
    all_preds_hb = []
    all_gt_hb = []

    with torch.no_grad():
        for imgs, demographic, hb_vals, bin_idx in tqdm(loader, desc="eval", leave=False):
            imgs = imgs.to(device)
            demographic = demographic.to(device)
            hb_vals = hb_vals.to(device)
            bin_idx = bin_idx.to(device).long()

            hb_pred, bin_logits = model(imgs, demographic)

            loss_reg = criterion_reg(hb_pred, hb_vals)
            loss_bin = criterion_focal(bin_logits, bin_idx)
            loss = loss_reg + ALPHA * loss_bin

            running_loss += loss.item() * imgs.size(0)

            all_preds_hb.extend(hb_pred.cpu().numpy().tolist())
            all_gt_hb.extend(hb_vals.cpu().numpy().tolist())

    epoch_loss = running_loss / len(loader.dataset)
    mae = mean_absolute_error(all_gt_hb, all_preds_hb)
    return epoch_loss, {"mae": mae}

In [ ]:
def main():
    # transforms
    base_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])

    # Load datasets
    train_dataset = EyesDefyDataset(train_csv_path, transform=base_transforms)
    valid_dataset = EyesDefyDataset(valid_csv_path, transform=base_transforms)
    print(len(train_dataset),len(valid_dataset))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=2, pin_memory=True)

    val_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                            shuffle=False, num_workers=2, pin_memory=True)

    # Model
    model = BPANet_Simple(pretrained=True, feature_dim=1000).to(DEVICE)

    criterion_reg = nn.MSELoss()
    criterion_focal = FocalLoss(gamma=2.0)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)#, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=4
    )

    best_val_mae = 999

    # Training loop
    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\nEpoch {epoch}/{NUM_EPOCHS}")

        train_loss, train_metrics = train_one_epoch(
            model, train_loader, optimizer, criterion_reg, criterion_focal, DEVICE
        )

        val_loss, val_metrics = eval_model(
            model, val_loader, criterion_reg, criterion_focal, DEVICE
        )

        scheduler.step(val_loss)

        print(f"Train Loss {train_loss:.4f} | MAE {train_metrics['mae']:.3f}")
        print(f"Val   Loss {val_loss:.4f} | MAE {val_metrics['mae']:.3f}")

        if val_metrics["mae"] < best_val_mae:
            best_val_mae = val_metrics["mae"]
            save_path = f"best_model.pth"

            torch.save({
                'epoch': epoch,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'val_metrics': val_metrics,
                'train_metrics': train_metrics
            }, save_path)

            print(f">>> Saved BEST model → {save_path}")

    print(f"\nTraining complete. Best MAE = {best_val_mae:.3f}")

if __name__ == "__main__": main()

159 42
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]



Epoch 1/140


train:   0%|          | 0/10 [00:00<?, ?it/s]

eval:   0%|          | 0/3 [00:00<?, ?it/s]

...

train:   0%|          | 0/10 [00:00<?, ?it/s]

eval:   0%|          | 0/3 [00:00<?, ?it/s]

Train Loss 3.2122 | MAE 0.988
Val   Loss 3.8761 | MAE 1.077
>>> Saved BEST model → best_model.pth

Epoch 4/140


train:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

##Five fold bad

In [ ]:
from sklearn.model_selection import KFold

def main():
    # Basic transforms
    base_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])

    full_dataset = EyesDefyDataset(DATA_CSV, transform=base_transforms)
    n = len(full_dataset)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(range(n))):
        print(f"\n============================")
        print(f"      Fold {fold+1} / 5")
        print(f"============================")

        # Subset samplers
        train_subset = torch.utils.data.Subset(full_dataset, train_idx)
        val_subset   = torch.utils.data.Subset(full_dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE,
                                  shuffle=True, num_workers=2, pin_memory=True)
        val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE,
                                shuffle=False, num_workers=2, pin_memory=True)

        # New model every fold
        model = BPANet_Simple(pretrained=True, feature_dim=1000)
        model = model.to(DEVICE)

        criterion_reg = nn.MSELoss()
        criterion_focal = FocalLoss(gamma=2.0)
        optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=4
        )

        best_val_mae = 999

        # Train
        for epoch in range(1, NUM_EPOCHS + 1):
            print(f"\n[Fold {fold+1}] Epoch {epoch}/{NUM_EPOCHS}")

            train_loss, train_metrics = train_one_epoch(
                model, train_loader, optimizer, criterion_reg, criterion_focal, DEVICE
            )

            val_loss, val_metrics = eval_model(
                model, val_loader, criterion_reg, criterion_focal, DEVICE
            )

            scheduler.step(val_loss)

            print(f"Train Loss {train_loss:.4f} | MAE {train_metrics['mae']:.3f}")
            print(f"Val   Loss {val_loss:.4f} | MAE {val_metrics['mae']:.3f}")

            # Save best fold model
            if val_metrics["mae"] < best_val_mae:
                best_val_mae = val_metrics["mae"]

                save_path = f"fold_{fold+1}_best.pth"
                torch.save({
                    'fold': fold+1,
                    'epoch': epoch,
                    'model_state': model.state_dict(),
                    'optimizer_state': optimizer.state_dict(),
                    'val_metrics': val_metrics,
                    'train_metrics': train_metrics
                }, save_path)

                print(f">>> Saved best model for fold {fold+1} → {save_path}")

        # Track fold result
        fold_results.append(best_val_mae)

    # At end:
    print("\n============================")
    print("  5-FOLD RESULTS")
    print("============================")
    for f, mae in enumerate(fold_results):
        print(f"Fold {f+1}: MAE = {mae:.3f}")

    print(f"\nAverage MAE: {np.mean(fold_results):.3f}")


In [ ]:
if __name__ == "__main__": main()


      Fold 1 / 5

[Fold 1] Epoch 1/140


train:   0%|          | 0/138 [00:00<?, ?it/s]

eval:   0%|          | 0/35 [00:00<?, ?it/s]

Train Loss 0.8539 | MAE 0.114
Val   Loss 0.2254 | MAE 0.058
>>> Saved best model for fold 1 → fold_1_best.pth

[Fold 1] Epoch 2/140


train:   0%|          | 0/138 [00:00<?, ?it/s]

...

eval:   0%|          | 0/35 [00:00<?, ?it/s]

Train Loss 0.0003 | MAE 0.006
Val   Loss 0.0248 | MAE 0.009

[Fold 1] Epoch 28/140


train:   0%|          | 0/138 [00:00<?, ?it/s]

eval:   0%|          | 0/35 [00:00<?, ?it/s]

KeyboardInterrupt: 

##Full train valid SPLIT TRAINING

In [ ]:
# -------------------------
# Main training scaffolding
# -------------------------
def main():
    # Only basic preprocessing (augmentations already done using albumentations)
    base_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])

    dataset = EyesDefyDataset(DATA_CSV, transform=base_transforms)
    n = len(dataset)
    if n < 10:
        raise ValueError("Dataset too small or CSV path incorrect.")
    # 80-20 split (shuffle)
    val_size = int(0.2 * n)
    train_size = n - val_size
    train_set, val_set = random_split(dataset, [train_size, val_size])
    # Ensure val uses val_transforms (random_split returns Subset; replace transform)
    train_set.dataset.transform = base_transforms
    val_set.dataset.transform = base_transforms

    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = BPANet_Simple(pretrained=True, feature_dim=1000)
    model = model.to(DEVICE)

    # Losses + optimizer
    criterion_reg = nn.MSELoss()
    criterion_focal = FocalLoss(gamma=2.0)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4, verbose=True)

    for epoch in range(1, NUM_EPOCHS+1):
        print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
        train_loss, train_metrics = train_one_epoch(model, train_loader, optimizer, criterion_reg, criterion_focal, DEVICE)
        val_loss, val_metrics = eval_model(model, val_loader, criterion_reg, criterion_focal, DEVICE)
        scheduler.step(val_loss)

        print(f"Train loss: {train_loss:.4f} | mae {train_metrics['mae']:.3f}")
        print(f"Val   loss: {val_loss:.4f} | mae {val_metrics['mae']:.3f}")

        # Save best by validation F1
        best_val_mae = 999
        if val_metrics["mae"] < best_val_mae:
            savepath = "bpanet_simple_best.pth"
            torch.save({
                'epoch': epoch,
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'val_metrics': val_metrics,
                'train_metrics': train_metrics
            }, savepath)
            print(f">>> Saved best model to {savepath} (valMAE {best_val_mae:.4f})")

    print("Training complete.")

if __name__ == "__main__":
    main()